# Agenda: Intro to decorators

1. What are decorators?
2. Decorating functions
3. Outer functions and decorators

In [1]:
def a():
    return f'a\n'

def b():
    return f'b\n'

print(a())
print(b())

a

b



In [5]:
# what if there's a new specification -- or a new demand from our customers
# all output needs to have a dashed line both before and after its output

lines = '-' * 60 + '\n'

def a():
    return f'{lines}a\n{lines}'

def b():
    return f'{lines}b\n{lines}'

print(a())
print(b())

------------------------------------------------------------
a
------------------------------------------------------------

------------------------------------------------------------
b
------------------------------------------------------------



# What's the problem?

We need to DRY up our code.  DRY (don't repeat yourself)

This means that we have to change every single one of our functions.  Each function is going to be changed in precisely the same way. Moreover, if the specifications change again, we'll need to change each of our functions **again**.  This is a huge waste of time and energy and effort.

So, what can we do?

In [6]:
# Remember that functions in Python are objects, just like every other object.
# We can pass a function as an argument to another function.

lines = '-' * 60 + '\n'

def with_lines(func):   # func is a function passed to with_lines
    return f'{lines}{func()}{lines}'

def a():
    return f'a\n'

def b():
    return f'b\n'

print(with_lines(a))
print(with_lines(b))

------------------------------------------------------------
a
------------------------------------------------------------

------------------------------------------------------------
b
------------------------------------------------------------



# Inner functions!

Remember:

- Functions are objects, and can thus be passed as arguments to other functions, and can also be returned by functions
- If we define a variable inside of a function, it is a local variable in that function.
- `def` defines a variable (as well as creating a function object).

In [7]:
# Putting all of this together, we can say:

def outer():
    def inner():
        return 'I am in inner!'
    return inner

f = outer()

In [8]:
f

<function __main__.outer.<locals>.inner()>

In [9]:
f()

'I am in inner!'

In [10]:
def outer(x):
    def inner(y):
        return f'I am in inner, {x=} and {y=}!'  # shows variable name + value
    return inner

f = outer(10)
f(5)

'I am in inner, x=10 and y=5!'

In [11]:
f(6)

'I am in inner, x=10 and y=6!'

In [12]:
g = outer(20)
g(5)

'I am in inner, x=20 and y=5!'

In [13]:
# let's redefine with_lines such that it returns not a string, but a FUNCTION

lines = '-' * 60 + '\n'

def with_lines(func):   # func is a function passed to with_lines
    def wrapper():
        return f'{lines}{func()}{lines}'
    return wrapper

def a():
    return f'a\n'

# three functions:
# (1) original a, a function that we defined
# (2) with_lines, a function that we defined
# (3) the new a, which is the result of calling with_lines(a), the value being wrapper -- the inner from with_lines

a = with_lines(a)    # when we call a now, we're really calling wrapper, defined when func is (the original) a

def b():
    return f'b\n'
b = with_lines(b)  # when we call b, we're really calling wrapper, defined when func is (the original) b

print(a())
print(b())

------------------------------------------------------------
a
------------------------------------------------------------

------------------------------------------------------------
b
------------------------------------------------------------



In [14]:
# let's redefine with_lines such that it returns not a string, but a FUNCTION

lines = '-' * 60 + '\n'

def with_lines(func):   # func is a function passed to with_lines
    def wrapper():
        return f'{lines}{func()}{lines}'
    return wrapper

@with_lines   # exactly the same as "a = with_lines(a)", if put after "def a"
def a():
    return f'a\n'

@with_lines
def b():
    return f'b\n'

print(a())
print(b())

------------------------------------------------------------
a
------------------------------------------------------------

------------------------------------------------------------
b
------------------------------------------------------------



In [18]:
# let's redefine with_lines such that it returns not a string, but a FUNCTION

lines = '-' * 60 + '\n'

def with_lines(func):   # func is a function passed to with_lines
    def wrapper(*args):  # capture all positional args in args
        return f'{lines}{func(*args)}{lines}'   # pass along any positional arguments to func
    return wrapper

@with_lines   # exactly the same as "a = with_lines(a)", if put after "def a"
def a():
    return f'a\n'

@with_lines
def b():
    return f'b\n'

@with_lines
def add(x, y):
    return x + y

print(a())
print(b())
print(add(3, 5))

------------------------------------------------------------
a
------------------------------------------------------------

------------------------------------------------------------
b
------------------------------------------------------------

------------------------------------------------------------
8------------------------------------------------------------



# To write a decorator

1. Write an outer function that takes one argument, a function (`func`).
2. The body of the outer function will define a new function, traditionally called `wrapper`.  
3. `wrapper` should take `*args` as an argument.
4. The return value from `wrapper` will replace the return value from the original (decorated) function.  It can include the orignial function's value, but it can replace it with something else or add to it or modify it.
5. The outer function returns `wrapper`.

# Where might we use a decorator?

- Redirect `stdout` (or `stderr`) for logging to a file
- Check permissions before running a function
- Check the time/date before running a function
- Check the timing of a function's execution, and log it
- Transform arguments before passing them to a function
- Trns